In [3]:
pip install requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [72]:
import requests
import os
from dotenv import load_dotenv
print("시작")
# .env 파일에서 환경 변수 로드
load_dotenv()

NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

# 공통 출력 함수
def print_book_info(book):
    print(f"제목: {book['title']}")
    print(f"저자: {book['author']}")
    print(f"출판사: {book['publisher']}")
    print(f"출간일: {book['pubdate']}")
    print(f"ISBN: {book['isbn']}")
    print(f"링크: {book['link']}")
    print('-' * 40)

# 네이버 책 검색 API 호출 함수
def search_books(query):
    url = 'https://openapi.naver.com/v1/search/book.json'
    headers = {
        'X-Naver-Client-Id': NAVER_CLIENT_ID,
        'X-Naver-Client-Secret': NAVER_CLIENT_SECRET
    }
    payload = {
        'query': query,
        'display': 30,
        'sort': 'sim'
    }

    res = requests.get(url, headers=headers, params=payload)

    if res.status_code == 200:
        items = res.json().get('items', [])
        if not items:
            print("검색 결과가 없습니다.")
        else:
            for book in items:
                print_book_info(book)
    else:
        print(f"오류 발생: {res.status_code} - {res.text}")

search_books("파이썬")

시작
제목: 혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임, 맵, AI 챗봇 만들기|저자 직강 유튜브 강의, 오픈 채팅 제공)
저자: 문현일
출판사: 한빛미디어
출간일: 20250214
ISBN: 9791169213271
링크: https://search.shopping.naver.com/book/catalog/52859338841
----------------------------------------
제목: 혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)
저자: 윤인성
출판사: 한빛미디어
출간일: 20220601
ISBN: 9791162245651
링크: https://search.shopping.naver.com/book/catalog/32507605957
----------------------------------------
제목: Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)
저자: 박응용
출판사: 이지스퍼블리싱
출간일: 20230615
ISBN: 9791163034735
링크: https://search.shopping.naver.com/book/catalog/40354085633
----------------------------------------
제목: 밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)
저자: 사이토 고키
출판사: 한빛미디어
출간일: 20250124
ISBN: 9791169213387
링크: https://search.shopping.naver.com/book/catalog/52615553184
----------------------------------------
제목: 파이썬
저자: 홍의경
출판사: 생능출판
출간일: 20220309
ISBN: 9788970506784
링크: https://search.shopping.naver.com/book/catalog/32460987987
----------------

In [86]:
import requests
import os
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

def get_book_price_from_shop(title):
    url = "https://openapi.naver.com/v1/search/shop.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {
        "query": title,
        "display": 1,  # 가장 관련도 높은 결과 1개만
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        items = response.json().get('items', [])
        if items:
            try:
                price = int(items[0]['lprice'])  # 최저가
                return f"{price:,}원"
            except:
                return "가격 정보 없음"
    return "가격 정보 없음"
    
# 책 가격 크롤링 함수 (네이버 쇼핑)
def get_book_price(title):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    search_url = f"https://search.shopping.naver.com/search/all?query={title}"
    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        return "가격 정보 없음"

    soup = BeautifulSoup(response.text, "html.parser")
    price_tag = soup.select_one(".price_num__S2p_v")  # 네이버 쇼핑 첫 결과의 가격
    if price_tag:
        price = re.sub(r"[^\d]", "", price_tag.text)
        return f"{int(price):,}원"
    else:
        return "가격 정보 없음"

# 도서 출력 함수
def print_book_list(books):
    print("=== 도서 목록 ===")
    for idx, book in enumerate(books, start=1):
        title = book.get('title', '').replace('<b>', '').replace('</b>', '')
        author = book.get('author', '').strip()
        publisher = book.get('publisher', '').strip()
        pubdate = book.get('pubdate', '')
        description = book.get('description', '').strip()

        # 출판일 포맷 변경
        if len(pubdate) == 8:
            pubdate = datetime.strptime(pubdate, "%Y%m%d").strftime("%Y-%m-%d")
        else:
            pubdate = "날짜 정보 없음"

        # 설명 자르기
        if len(description) > 100:
            description = description[:100].rstrip() + "..."

        # 크롤링으로 가격 가져오기
        price = get_book_price(title)

        print(f"{idx}. {title}")
        print(f"   저자: {author}")
        print(f"   출판사: {publisher}")
        print(f"   출판일: {pubdate}")
        price = get_book_price_from_shop(title)
        print(f"   가격: {price}")
        print(f"   설명: {description}")
        print()


# 책 검색 함수
def search_books(query):
    url = "https://openapi.naver.com/v1/search/book.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": 3,  # 테스트용으로 3권만
        "start": 1,
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        books = response.json().get('items', [])
        if books:
            print_book_list(books)
        else:
            print("검색 결과가 없습니다.")
    else:
        print(f"API 호출 실패: {response.status_code}")
        print("응답 메시지:", response.text)





In [88]:
search_books("파이썬")


=== 도서 목록 ===
1. 혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임, 맵, AI 챗봇 만들기|저자 직강 유튜브 강의, 오픈 채팅 제공)
   저자: 문현일
   출판사: 한빛미디어
   출판일: 2025-02-14
   가격: 가격 정보 없음
   설명: “파이썬을 배워서 어디에 활용할 수 있을지 궁금했나요?”
나의 신용카드 내역 분석, 우리 아파트 실거래가 시각화해보기, 맛집 지도 웹 앱 만들기 등 
생활 속  15가지 파이썬 프...

2. 혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)
   저자: 윤인성
   출판사: 한빛미디어
   출판일: 2022-06-01
   가격: 29,360원
   설명: 혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서

『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었습니다. 프로그래밍이 정말 처음인 입문자도 따...

3. Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)
   저자: 박응용
   출판사: 이지스퍼블리싱
   출판일: 2023-06-15
   가격: 가격 정보 없음
   설명: 프로그래밍 분야 8년 연속 베스트셀러!
《Do it! 점프 투 파이썬》 전면 개정 2판 출시!

중고등학생도, 비전공자도, 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개...



In [92]:
#1. 질문 :  검색어로  찾은  책 목록 출력하기
import requests
import os
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

# 공통 headers
headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

# 가격 조회 함수 (네이버 쇼핑 API)
def get_book_price_from_shop(title):
    url = "https://openapi.naver.com/v1/search/shop.json"
    clean_title = re.sub(r"\(.*?\)", "", title).strip()
    for keyword in [clean_title + " 책", clean_title]:
        params = {
            "query": keyword,
            "display": 5,
            "sort": "sim"
        }

        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            items = res.json().get('items', [])
            prices = []
            for item in items:
                try:
                    price = int(item['lprice'])
                    prices.append(price)
                except:
                    continue
            if prices:
                return f"{min(prices):,}원"
    return "가격 정보 없음"

# 날짜 포맷팅 함수
def format_pubdate(pubdate):
    try:
        if len(pubdate) == 8:
            return datetime.strptime(pubdate, "%Y%m%d").strftime("%Y-%m-%d")
    except:
        pass
    return "날짜 정보 없음"

# 책 목록 출력 함수
def print_books(items):
    print("=== 도서 목록 ===")
    for i, book in enumerate(items, 1):
        title = book['title'].replace('<b>', '').replace('</b>', '')
        author = book['author'].strip()
        publisher = book['publisher'].strip()
        pubdate = format_pubdate(book['pubdate'])
        description = book['description'].strip().replace('\n', ' ')
        if len(description) > 100:
            description = description[:100].rstrip() + "..."
        price = get_book_price_from_shop(title)

        print(f"{i}. {title}")
        print(f"   저자: {author}")
        print(f"   출판사: {publisher}")
        print(f"   출판일: {pubdate}")
        print(f"   가격: {price}")
        print(f"   설명: {description}")
        print()

# 책 검색 함수
def search_books(query):
    url = "https://openapi.naver.com/v1/search/book.json"
    payload = {
        "query": query,
        "display": 5,
        "sort": "sim"
    }
    res = requests.get(url, headers=headers, params=payload)
    if res.status_code == 200:
        items = res.json().get("items", [])
        if items:
            print_books(items)
        else:
            print("검색 결과가 없습니다.")
    else:
        print(f"API 호출 실패: {res.status_code}")
        print("응답 메시지:", res.text)

# 실행
search_books("파이썬")

=== 도서 목록 ===
1. 혼자 만들면서 공부하는 파이썬 (37개 라이브러리와 API로 보고서, 차트, 게임, 맵, AI 챗봇 만들기|저자 직강 유튜브 강의, 오픈 채팅 제공)
   저자: 문현일
   출판사: 한빛미디어
   출판일: 2025-02-14
   가격: 19,800원
   설명: “파이썬을 배워서 어디에 활용할 수 있을지 궁금했나요?” 나의 신용카드 내역 분석, 우리 아파트 실거래가 시각화해보기, 맛집 지도 웹 앱 만들기 등  생활 속  15가지 파이썬 프...

2. 혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)
   저자: 윤인성
   출판사: 한빛미디어
   출판일: 2022-06-01
   가격: 18,900원
   설명: 혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서  『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었습니다. 프로그래밍이 정말 처음인 입문자도 따...

3. Do it! 점프 투 파이썬 (중학생도 첫날부터 실습하는 초고속 입문서)
   저자: 박응용
   출판사: 이지스퍼블리싱
   출판일: 2023-06-15
   가격: 19,800원
   설명: 프로그래밍 분야 8년 연속 베스트셀러! 《Do it! 점프 투 파이썬》 전면 개정 2판 출시!  중고등학생도, 비전공자도, 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개...

4. 밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)
   저자: 사이토 고키
   출판사: 한빛미디어
   출판일: 2025-01-24
   가격: 23,400원
   설명: 딥러닝 분야 부동의 베스트셀러! 머리로 이해하고 손으로 익히는 가장 쉬운 딥러닝 입문서  이 책은 딥러닝의 핵심 개념을 ‘밑바닥부터’ 구현해보며 기초를 한 걸음씩 탄탄하게 다질 수...

5. 파이썬
   저자: 홍의경
   출판사: 생능출판
   출판일: 2022-03-09
   가격: 40,940원
   설명: 코딩 실

In [94]:
#2. 질문 :  검색어로  찾은  책 목록 중에서 가격이 2만원 이상인 책만 출력하기

import requests
import os
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

# 네이버 쇼핑 API로 가격 조회 (문자형, 정수형 둘 다 반환)
def get_book_price_from_shop(title):
    url = "https://openapi.naver.com/v1/search/shop.json"
    clean_title = re.sub(r"\(.*?\)", "", title).strip()

    for keyword in [clean_title + " 책", clean_title]:
        params = {
            "query": keyword,
            "display": 5,
            "sort": "sim"
        }
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            items = res.json().get('items', [])
            prices = []
            for item in items:
                try:
                    price = int(item['lprice'])
                    if 5000 <= price <= 100000:
                        prices.append(price)
                except:
                    continue
            if prices:
                min_price = min(prices)
                return f"{min_price:,}원", min_price
    return "가격 정보 없음", 0

# 출판일 포맷
def format_pubdate(pubdate):
    try:
        if len(pubdate) == 8:
            return datetime.strptime(pubdate, "%Y%m%d").strftime("%Y-%m-%d")
    except:
        pass
    return "날짜 정보 없음"

# 책 목록 출력 함수 (2만원 이상만 필터링)
def print_expensive_books(items):
    print("=== 가격이 20,000원 이상인 도서 목록 ===")
    count = 1
    for book in items:
        title = book['title'].replace('<b>', '').replace('</b>', '')
        author = book['author'].strip()
        publisher = book['publisher'].strip()
        pubdate = format_pubdate(book['pubdate'])
        description = book['description'].strip().replace('\n', ' ')
        if len(description) > 100:
            description = description[:100].rstrip() + "..."

        price_str, price_int = get_book_price_from_shop(title)

        if price_int >= 20000:
            print(f"{count}. {title}")
            print(f"   저자: {author}")
            print(f"   출판사: {publisher}")
            print(f"   출판일: {pubdate}")
            print(f"   가격: {price_str}")
            print(f"   설명: {description}")
            print()
            count += 1

# 책 검색 함수
def search_books(query):
    url = "https://openapi.naver.com/v1/search/book.json"
    payload = {
        "query": query,
        "display": 10,
        "sort": "sim"
    }
    res = requests.get(url, headers=headers, params=payload)
    if res.status_code == 200:
        items = res.json().get("items", [])
        if items:
            print_expensive_books(items)
        else:
            print("검색 결과가 없습니다.")
    else:
        print(f"API 호출 실패: {res.status_code}")
        print("응답 메시지:", res.text)

# 실행
search_books("파이썬")

=== 가격이 20,000원 이상인 도서 목록 ===
1. 밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현)
   저자: 사이토 고키
   출판사: 한빛미디어
   출판일: 2025-01-24
   가격: 23,400원
   설명: 딥러닝 분야 부동의 베스트셀러! 머리로 이해하고 손으로 익히는 가장 쉬운 딥러닝 입문서  이 책은 딥러닝의 핵심 개념을 ‘밑바닥부터’ 구현해보며 기초를 한 걸음씩 탄탄하게 다질 수...

2. 파이썬
   저자: 홍의경
   출판사: 생능출판
   출판일: 2022-03-09
   가격: 40,940원
   설명: 코딩 실력을 키우는 지름길은 실습이다.  2008년 즈음에 검인정 중학교 교과서 정보1, 정보2, 정보3을 공저로 저술하였습니다. 그 당시에 8종류의 교과서가 검인정에 합격하였는데...

3. CODING BASICS PYTHON (파이썬)
   저자: 김상민^장성식^김일태
   출판사: 렉스미디어닷넷
   출판일: 2022-02-10
   가격: 29,200원
   설명: 파이썬으로 다지는 프로그래밍의 기초  ㆍ 다양한 예제를 활용하여 초보자도 쉽게 프로그래밍의 기본을 익힐 수 있는 파이썬 입문서입니다. ㆍ 문법을 사전식으로 나열하지 않고 기능적인...

4. 파이썬 (제2판)
   저자: 염기원^오지영
   출판사: 북두
   출판일: 2022-01-20
   가격: 40,940원
   설명: 책의 전반부는 비전공자의 입장에서 비전공자를 위한 파이썬 프로그래밍의 기초적인 내용으로 구성되어 있다. 프로그래밍에 익숙하거나 파이썬 경험이 있는 독자들은 곧바로 책의 후반부를 공...

5. 파이썬
   저자: 천인국
   출판사: 인피니티북스
   출판일: 2017-08-30
   가격: 40,940원
   설명: 컴퓨팅 사고(Computational Thinking)는 자넷 윙 교수가 2006년에 컴퓨터 학술지에서 언급한 이후로 컴퓨터 교육 분야에서 상당한 이슈가 되고 있는 용어이다. 컴퓨

In [96]:
3. 질문 :  검색어로  찾은  책 목록 중에서 출판사가 인피니티북스인 책만 출력하기
import requests
import os
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

# 네이버 쇼핑 API로 가격 조회 (문자형, 정수형 둘 다 반환)
def get_book_price_from_shop(title):
    url = "https://openapi.naver.com/v1/search/shop.json"
    clean_title = re.sub(r"\(.*?\)", "", title).strip()

    for keyword in [clean_title + " 책", clean_title]:
        params = {
            "query": keyword,
            "display": 5,
            "sort": "sim"
        }
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            items = res.json().get('items', [])
            prices = []
            for item in items:
                try:
                    price = int(item['lprice'])
                    if 5000 <= price <= 100000:
                        prices.append(price)
                except:
                    continue
            if prices:
                min_price = min(prices)
                return f"{min_price:,}원", min_price
    return "가격 정보 없음", 0

# 출판일 포맷
def format_pubdate(pubdate):
    try:
        if len(pubdate) == 8:
            return datetime.strptime(pubdate, "%Y%m%d").strftime("%Y-%m-%d")
    except:
        pass
    return "날짜 정보 없음"

# 출판사가 인피니티북스인 도서만 출력
def print_infinitybooks_only(items):
    print("=== 인피니티북스 출판 도서 목록 ===")
    count = 1
    for book in items:
        title = book['title'].replace('<b>', '').replace('</b>', '')
        author = book['author'].strip()
        publisher = book['publisher'].strip()
        pubdate = format_pubdate(book['pubdate'])
        description = book['description'].strip().replace('\n', ' ')
        if len(description) > 100:
            description = description[:100].rstrip() + "..."

        # 출판사 필터
        if publisher != "인피니티북스":
            continue

        price_str, _ = get_book_price_from_shop(title)

        print(f"{count}. {title}")
        print(f"   저자: {author}")
        print(f"   출판사: {publisher}")
        print(f"   출판일: {pubdate}")
        print(f"   가격: {price_str}")
        print(f"   설명: {description}")
        print()
        count += 1

# 책 검색 함수
def search_books(query):
    url = "https://openapi.naver.com/v1/search/book.json"
    payload = {
        "query": query,
        "display": 30,
        "sort": "sim"
    }
    res = requests.get(url, headers=headers, params=payload)
    if res.status_code == 200:
        items = res.json().get("items", [])
        if items:
            print_infinitybooks_only(items)
        else:
            print("검색 결과가 없습니다.")
    else:
        print(f"API 호출 실패: {res.status_code}")
        print("응답 메시지:", res.text)

# 실행
search_books("파이썬")

=== 인피니티북스 출판 도서 목록 ===
1. 파이썬
   저자: 천인국
   출판사: 인피니티북스
   출판일: 2017-08-30
   가격: 40,940원
   설명: 컴퓨팅 사고(Computational Thinking)는 자넷 윙 교수가 2006년에 컴퓨터 학술지에서 언급한 이후로 컴퓨터 교육 분야에서 상당한 이슈가 되고 있는 용어이다. 컴퓨...

2. 파이썬 플러스
   저자: 최희식
   출판사: 인피니티북스
   출판일: 2024-07-31
   가격: 41,000원
   설명: 파이썬 최신 버전 반영! 파이썬을 마스터하고 싶은 학습자들의 필독서!  『핵심코어! 파이썬 프로그래밍』이 더욱 알찬 구성으로 업데이트되어 새로운 책으로 돌아왔다. 『파이썬 플러스』...



In [98]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
}

# 네이버 쇼핑 API로 가격 조회
def get_book_price_from_shop(title):
    url = "https://openapi.naver.com/v1/search/shop.json"
    clean_title = re.sub(r"\(.*?\)", "", title).strip()

    for keyword in [clean_title + " 책", clean_title]:
        params = {
            "query": keyword,
            "display": 5,
            "sort": "sim"
        }
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            items = res.json().get('items', [])
            prices = []
            for item in items:
                try:
                    price = int(item['lprice'])
                    if 5000 <= price <= 100000:
                        prices.append(price)
                except:
                    continue
            if prices:
                min_price = min(prices)
                return f"{min_price:,}원", min_price
    return "가격 정보 없음", 0

# 출판일 포맷
def format_pubdate(pubdate):
    try:
        if len(pubdate) == 8:
            return datetime.strptime(pubdate, "%Y%m%d").strftime("%Y-%m-%d")
    except:
        pass
    return "날짜 정보 없음"

# 검색 결과 JSON 저장
def save_books_to_json(items, filename="data/books.json"):
    books_data = []

    for book in items:
        title = book['title'].replace('<b>', '').replace('</b>', '')
        author = book['author'].strip()
        publisher = book['publisher'].strip()
        pubdate = format_pubdate(book['pubdate'])
        description = book['description'].strip().replace('\n', ' ')
        if len(description) > 100:
            description = description[:100].rstrip() + "..."

        price_str, price_int = get_book_price_from_shop(title)

        book_info = {
            "title": title,
            "author": author,
            "publisher": publisher,
            "pubdate": pubdate,
            "price": price_str,
            "description": description
        }
        books_data.append(book_info)

    # 디렉터리 생성
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    # JSON 저장
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(books_data, f, ensure_ascii=False, indent=2)

    print(f"✅ {len(books_data)}권의 책 정보를 '{filename}'에 저장했습니다.")

# 책 검색 함수
def search_books(query):
    url = "https://openapi.naver.com/v1/search/book.json"
    payload = {
        "query": query,
        "display": 30,
        "sort": "sim"
    }
    res = requests.get(url, headers=headers, params=payload)
    if res.status_code == 200:
        items = res.json().get("items", [])
        if items:
            save_books_to_json(items)
        else:
            print("검색 결과가 없습니다.")
    else:
        print(f"API 호출 실패: {res.status_code}")
        print("응답 메시지:", res.text)

# 실행
search_books("파이썬")

✅ 30권의 책 정보를 'data/books.json'에 저장했습니다.
